In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from sklearn.metrics import matthews_corrcoef

y_true = [1, 1, 1, 0]
y_pred = [1, 0, 1, 1]

score = matthews_corrcoef(y_true, y_pred)

score

-0.3333333333333333

In [3]:
score = matthews_corrcoef(y_true, y_true)

score

1.0

In [4]:
y_pred = [0, 0, 0, 1]

score = matthews_corrcoef(y_true, y_pred)

score

-1.0

In [5]:
import os 

path = 'data/eopatches/train'
patches = os.listdir(path)
len(patches)

100

In [6]:
from src.utils import generate_mask_with_mean_ndvi

patch = patches[1]

mask = generate_mask_with_mean_ndvi(path, patch)

mask.shape, mask.dtype, mask.max(), mask.min()

((2000, 2000), dtype('bool'), True, False)

In [7]:
from src.utils import get_npy

target = get_npy(path, patch, 'mask_timeless/CULTIVATED')[...,0]

target.shape, target.dtype, target.max(), target.min()

((2000, 2000), dtype('bool'), True, False)

In [8]:
import matplotlib.pyplot as plt 

fig = plt.figure()
ax = plt.subplot(1, 2, 1)
ax.imshow(target)
ax.axis('off')
ax = plt.subplot(1, 2, 2)
ax.imshow(mask)
ax.axis('off')
plt.tight_layout()
plt.plot()

[]

In [9]:
score = matthews_corrcoef(target.ravel(), mask.ravel())

score

0.3066223562022115

In [10]:
from tqdm import tqdm 

def compute_mask(patch):
    mask = generate_mask_with_mean_ndvi(path, patch)
    target = get_npy(path, patch, 'mask_timeless/CULTIVATED')[...,0]
    score = matthews_corrcoef(target.ravel(), mask.ravel())
    return score

In [11]:
import multiprocessing
from concurrent.futures import ProcessPoolExecutor
import warnings

warnings.simplefilter("ignore")

num_cores = multiprocessing.cpu_count()
with ProcessPoolExecutor(max_workers=num_cores) as pool:
    with tqdm(total=len(patches)) as progress:
        futures = []

        for patch in patches:
            future = pool.submit(compute_mask, patch)
            future.add_done_callback(lambda p: progress.update())
            futures.append(future)

        scores = []
        for future in futures:
            result = future.result()
            scores.append(result)

100%|██████████| 100/100 [00:32<00:00,  3.08it/s]


In [12]:
import numpy as np

np.mean(scores)

0.18062216769179792

In [13]:
import torch
from torchmetrics import MatthewsCorrcoef

gt = torch.tensor([1, 1, 0, 0])
preds = torch.tensor([0, 1, 0, 0])

matthews_corrcoef = MatthewsCorrcoef(num_classes=2)
matthews_corrcoef(preds, gt)

tensor(0.5774)

In [14]:
mask_t = torch.tensor(mask)
target_t = torch.tensor(target)

mask_t.shape, target_t.shape

(torch.Size([2000, 2000]), torch.Size([2000, 2000]))

In [15]:
matthews_corrcoef(mask_t, target_t)

tensor(0.3066)

In [16]:
mask_batch = torch.stack([mask_t]*3)
target_batch = torch.stack([target_t]*3)

mask_batch.shape, target_batch.shape

(torch.Size([3, 2000, 2000]), torch.Size([3, 2000, 2000]))

In [17]:
matthews_corrcoef(mask_batch, target_batch)

tensor(0.3066)

In [18]:
def get_tensor(patch):
    mask = generate_mask_with_mean_ndvi(path, patch)
    target = get_npy(path, patch, 'mask_timeless/CULTIVATED')[...,0]
    return mask, target

In [26]:
mask_tensors, target_tensors = [], []

for patch in tqdm(patches):
    mask, target = get_tensor(patch)
    mask_tensors.append(torch.tensor(mask))
    target_tensors.append(torch.tensor(target))

100%|██████████| 100/100 [02:04<00:00,  1.24s/it]


In [27]:
mask_batch = torch.stack(mask_tensors)
target_batch = torch.stack(target_tensors)

mask_batch.shape, target_batch.shape

(torch.Size([100, 2000, 2000]), torch.Size([100, 2000, 2000]))

In [28]:
matthews_corrcoef(mask_batch, target_batch)

tensor(0.1817)

In [30]:
device = "cuda:0"

mask_batch = mask_batch[:20].to(device)
target_batch = target_batch[:20].to(device)

matthews_corrcoef(mask_batch, target_batch)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!